# Persistence

Persistence (I/O) Example

Demonstrates saving and loading SparseTensor/DSparseTensor using safetensors format:
- SparseTensor.save() / SparseTensor.load()
- save_distributed() / load_partition()
- Cross-format loading

In [ ]:
sys.path.insert(0, '..')

import torch
import tempfile
import os


from torch_sla import SparseTensor, DSparseTensor, DSparseMatrix
from torch_sla.io import load_sparse_as_partition, load_distributed_as_sparse

## Helper Functions and Classes

In [ ]:
def create_2d_poisson(n: int, dtype=torch.float64):
    """Create 2D Poisson matrix."""
    N = n * n
    idx = torch.arange(N)
    i, j = idx // n, idx % n
    
    entries = [
        (idx, idx, torch.full((N,), 4.0, dtype=dtype)),
        (idx[i > 0], idx[i > 0] - n, torch.full(((i > 0).sum(),), -1.0, dtype=dtype)),
        (idx[i < n-1], idx[i < n-1] + n, torch.full(((i < n-1).sum(),), -1.0, dtype=dtype)),
        (idx[j > 0], idx[j > 0] - 1, torch.full(((j > 0).sum(),), -1.0, dtype=dtype)),
        (idx[j < n-1], idx[j < n-1] + 1, torch.full(((j < n-1).sum(),), -1.0, dtype=dtype)),
    ]
    
    row = torch.cat([e[0] for e in entries])
    col = torch.cat([e[1] for e in entries])
    val = torch.cat([e[2] for e in entries])
    
    return val, row, col, (N, N)